<a href="https://colab.research.google.com/github/niconielsen32/ObjectTracking/blob/main/ObjectTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get the requirements

In [ ]:
!pip install tf-centernet

In [ ]:

!pip install -r D:\Python\Projects\Ak\Akproj\requirements.txt

# Object Tracking

In [ ]:
import sys
import os
import centernet

from sort import Sort
from lib import VisTrack, show_video, create_video

import numpy as np
import PIL.Image
import cv2
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import shutil

In [ ]:
vt = VisTrack()

# Default: num_classes=80
obj = centernet.ObjectDetection(num_classes=80)

# num_classes=80 and weights_path=None: Pre-trained COCO model will be loaded.
obj.load_weights(weights_path=None)

## Predict on image

In [ ]:

p_img = PIL.Image.open('D:\Python\Projects\Ak\Akproj\people.jpg')
img = cv2.cvtColor(np.array(p_img), cv2.COLOR_BGR2RGB)

boxes, classes, scores = obj.predict(img)

vt.draw_bounding_boxes(p_img, boxes, classes, scores)

## Predict on live video

In [ ]:
vidcap = cv2.VideoCapture(0)

# Create the SORT tracker
sort = Sort(max_age=1, min_hits=3, iou_threshold=0.3)

# Create a window to display the output frame
cv2.namedWindow("Object Tracking", cv2.WINDOW_NORMAL)

folder_out = "Track"
if os.path.exists(folder_out):
    shutil.rmtree(folder_out)
os.makedirs(folder_out)

# Counter for frame numbering
i = 0

# Capture frames from the camera
while True:
    ret, frame = vidcap.read()
    if not ret:
        break

    boxes, classes, scores = obj.predict(frame)  # Assuming 'obj' is defined for object detection

    detections_in_frame = len(boxes)
    if detections_in_frame:
        # centernet will do detection on all the COCO classes. "person" is class number 0 
        idxs = np.where(classes == 0)[0]
        boxes = boxes[idxs]
        scores = scores[idxs]
        classes = classes[idxs]
    else:
        boxes = np.empty((0, 5))

    dets = np.hstack((boxes, scores[:, np.newaxis]))
    res = sort.update(dets)

    boxes_track = res[:, :-1]
    boxes_ids = res[:, -1].astype(int)

    # Convert the frame to a PIL Image
    p_frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if detections_in_frame:
        # Assuming 'vt.draw_bounding_boxes' is defined for drawing bounding boxes
        p_frame = vt.draw_bounding_boxes(p_frame, boxes_track, boxes_ids, scores)

    # Save the frame as an image
    p_frame.save(os.path.join(folder_out, f"{i:03d}.png"))

    # Convert the PIL Image back to a NumPy array for display
    frame_display = cv2.cvtColor(np.array(p_frame), cv2.COLOR_RGB2BGR)

    # Display the frame in the window
    cv2.imshow("Object Tracking", frame_display)

    i += 1

    # Check for the 'q' key to exit the loop and close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
vidcap.release()
cv2.destroyAllWindows()

## Tracking video output

In [ ]:


track_video_file = 'tracking.mp4'
create_video(frames_pattern='Track/%03d.png', video_file=track_video_file, framerate=30)

